In [2]:
import os
import numpy as np
import pandas as pd
import glob

import torch
from torch import nn
from torch.utils.data import TensorDataset,Dataset, DataLoader
from torch.optim import SGD, Adam

from torchvision import datasets
from torchvision import models
import torchvision.transforms as transforms

import cv2
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.ToTensor(),                   # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.1307], std=[0.3081])  # Normalize using FashionMNIST mean and std
])

trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.7MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 203kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.79MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 19.2MB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [4]:
R, C = 7,7
fig, ax = plt.subplots(R, C, figsize=(5,5))
for label_class, plot_row in enumerate(ax):
    for plot_cell in plot_row:
        plot_cell.grid(False); plot_cell.axis('off')
        ix = np.random.choice(1000)
        im, label = trainset[ix]
        plot_cell.imshow(im[0].cpu(), cmap='gray')
plt.tight_layout()

plt.savefig('data_snapshot.png', dpi=300)  # Save with high resolution
plt.close()  # Close the figure to free up memory

In [5]:
class SimpleModel(nn.Module):
    def __init__(self):
      super().__init__()
      self.simple_model = nn.Sequential(
        nn.Conv2d(1,32,3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(32,64,3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(64,128,3,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Flatten(),
        nn.Linear(128*3*3,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

    def forward(self, x):
        return self.simple_model(x)

# 4. Initialize the model, loss function, and optimizer
model = SimpleModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [6]:
from torchsummary import summary
summary(model, input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Conv2d-7            [-1, 128, 7, 7]          73,856
              ReLU-8            [-1, 128, 7, 7]               0
         MaxPool2d-9            [-1, 128, 3, 3]               0
          Flatten-10                 [-1, 1152]               0
           Linear-11                  [-1, 512]         590,336
             ReLU-12                  [-1, 512]               0
           Linear-13                   [-1, 10]           5,130
Total params: 688,138
Trainable params:

In [7]:
num_epochs = 6
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Track accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print statistics after each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {100 * correct/total:.2f}%")


Epoch [1/6], Loss: 0.4257, Accuracy: 84.31%
Epoch [2/6], Loss: 0.2632, Accuracy: 90.25%
Epoch [3/6], Loss: 0.2183, Accuracy: 91.79%
Epoch [4/6], Loss: 0.1864, Accuracy: 93.14%
Epoch [5/6], Loss: 0.1630, Accuracy: 93.87%
Epoch [6/6], Loss: 0.1386, Accuracy: 94.85%


In [17]:
def visualize_layer(layers,loader):
  image = visualize_image(loader)
  image = image.to(device)
  print(image.shape)
  for i, layer in enumerate(layers):
    outputs = layer(image[None])[0].to('cpu').detach()
    label = f'ConvLayer_{i}'

    # visualize image
    cols = int(np.ceil(np.sqrt(outputs.shape[0])))
    rows = int(np.ceil(outputs.shape[0] / cols))
    print(R,C)
    fig, ax = plt.subplots(rows, cols, figsize=(cols*2,rows*2))
    for idx, axis in enumerate(ax.flat):
      if idx >= outputs.shape[0]:
        axis.remove()
      else:
        axis.grid(False); axis.axis('off')
        axis.set_title(label + " Filter "+str(idx))
        axis.imshow(outputs[idx].cpu(), cmap='gray')

    plt.tight_layout()
    plt.savefig(f'conv_layer_{i}.png', dpi=300)  # Save with high resolution
    plt.close()  # Close the figure to free up memory

  #plt.show()

def visualize_dense_layer(layer, loader, label='Dense layer 1'):
  image = visualize_image(loader)
  outputs = layer(image[None])[0].detach()

  print(outputs.shape)
  plt.figure(figsize=(100,10))
  plt.imshow(outputs.cpu())


def visualize_image(loader):
  # take the random image
  size = len(loader.dataset)
  rnd =  np.random.choice(size)
  image, label = loader.dataset[rnd]
  plt.imshow(image[0].cpu())
  plt.axis('off')
  plt.savefig(f'ConvInputImage.png', dpi=300)  # Save with high resolution
  plt.close()
  #plt.show()
  return image

In [18]:
### Visualize first conv
first_layer=nn.Sequential(*list(model.simple_model.children())[:1]) # first conv layer
second_conv=nn.Sequential(*list(model.simple_model.children())[:4]) # mid conv layer
third_conv=nn.Sequential(*list(model.simple_model.children())[:7]) # last conv layer
layers = [first_layer, second_conv, third_conv]
visualize_layer(layers,trainloader)

torch.Size([1, 28, 28])
7 7
7 7
7 7


## Visualize flatten & dense layers

In [19]:
trainloader = DataLoader(trainset, batch_size=2498, shuffle=True, num_workers=2,drop_last=True)

In [20]:
x, y = next(iter(trainloader))

In [22]:
x2 = x[y==1] # taking any one class as an example
len(x2)

272

In [23]:
x2 = x2.view(len(x2),1,28,28)

In [25]:
flatten_layer=nn.Sequential(*list(model.simple_model.children())[:10])
flatten_layer_output = flatten_layer(x2.to(device)).detach()
print(flatten_layer_output.shape)
plt.figure(figsize=(100,10))
plt.imshow(flatten_layer_output.cpu())#,cmap='gray')

plt.savefig('Flatten_layer.png',dpi=300)
plt.close()

torch.Size([272, 1152])


In [26]:
first_dense=nn.Sequential(*list(model.simple_model.children())[:11])
flatten_layer_output = first_dense(x2.to(device)).detach()
#flatten_layer_output.shape

plt.figure(figsize=(100,10))
plt.imshow(flatten_layer_output.cpu(),cmap='gray')

plt.savefig('DenseLayer1.png',dpi=300)
plt.close()

In [27]:
### Visualize the last dense layer
last_dense=nn.Sequential(*list(model.simple_model.children())[:13])
flatten_layer_output = last_dense(x2.to(device)).detach()
# flatten_layer_output.shape
plt.figure(figsize=(100,10))
plt.imshow(flatten_layer_output.cpu(),cmap='gray')
plt.savefig('LastDenseLayer.png',dpi=300)
plt.close()